In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, GPT2LMHeadModel, GPT2Config, AdamW
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

/kaggle/input/tender-hack-dataset/tender-names/work_names
/kaggle/input/tender-hack-dataset/tender-names/uslugi_names
/kaggle/input/tender-hack-dataset/tender-names/products_names
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00002-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer_config.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model.bin.index.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/pytorch_model-00001-of-00002.bin
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/special_tokens_map.json
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/.gitattributes
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/tokenizer.model
/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1/generation_config.json


In [2]:
! pip install ipywidgets

In [10]:
import pickle

products_names = pickle.load(open('/kaggle/input/tender-hack-dataset/tender-names/products_names', 'rb'))
uslugi_names = pickle.load(open('/kaggle/input/tender-hack-dataset/tender-names/uslugi_names', 'rb'))
work_names = pickle.load(open('/kaggle/input/tender-hack-dataset/tender-names/work_names', 'rb'))

In [11]:
print(f'Товаров: {len(products_names)}, услуг: {len(uslugi_names)}, работ: {len(work_names)}')

Товаров: 86843, услуг: 9949, работ: 2652


In [12]:
import random

products_names_shuffle = random.sample(products_names, 2652 * 3)
uslugi_names_shuffle = random.sample(uslugi_names, 2652 * 2)
work_names_shuffle = random.sample(work_names, 2652)

In [13]:
print(f'Товаров: {len(products_names_shuffle)}, услуг: {len(uslugi_names_shuffle)}, работ: {len(work_names_shuffle)}')

Товаров: 7956, услуг: 5304, работ: 2652


In [14]:
df_products = pd.DataFrame({'Название': products_names_shuffle, 'Тип': 0, 'Флаг': 1})
df_uslugi = pd.DataFrame({'Название': uslugi_names_shuffle, 'Тип': 1, 'Флаг': 0})
df_work = pd.DataFrame({'Название': work_names_shuffle, 'Тип': 2, 'Флаг': 0})

types_df = pd.concat([df_products, df_uslugi, df_work], ignore_index=True)

In [15]:
types_df.head()

,Название,Тип,Флаг
0,"Доска-планшет BRAUBERG ""Black Jack"" с прижимом...",0,1
1,МФУ Катюша M240,0,1
2,Акридерм мазь д/наружн. прим 0.05% 30г туба №1...,0,1
3,Салфетки косметологические для обеззараживания...,0,1
4,Семечки От Мартина премиум 200г,0,1


In [16]:
from sklearn.utils import compute_class_weight

class_weights = compute_class_weight('balanced', classes=np.unique(types_df['Флаг']), y=types_df['Флаг'])
class_weights

array([1., 1.])

In [17]:
from gensim.models import Word2Vec
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

import torch
from transformers import DistilBertTokenizer, DistilBertModel
from transformers import RobertaTokenizer, RobertaModel

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', num_labels=3)
# model = DistilBertModel.from_pretrained('distilbert-base-uncased', num_labels=3)

tokenizer = RobertaTokenizer.from_pretrained('roberta-base', num_labels=3)
model = RobertaModel.from_pretrained('roberta-base', num_labels=3)

model.to(device)

def get_name_embedding(name, model, tokenizer):
    inputs = tokenizer(name, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    return torch.mean(outputs.last_hidden_state[:, 1:, :], dim=1).cpu().numpy()

types_df['embedding'] = types_df['Название'].apply(lambda x: get_name_embedding(x, model, tokenizer))

X = np.vstack(types_df['embedding'].values)
y = types_df['Флаг']

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.3, random_state=42)

train_pool = Pool(X_train, label=y_train)
test_pool = Pool(X_val, label=y_val)

model_1 = CatBoostClassifier(iterations=1400,
                             learning_rate=0.03,
                             depth=6,
                             loss_function='Logloss',
                             verbose=100,
                             class_weights=class_weights,
                             task_type='GPU')

model_1.fit(train_pool, eval_set=test_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6613198	test: 0.6620348	best: 0.6620348 (0)	total: 104ms	remaining: 2m 24s
100:	learn: 0.1966699	test: 0.2281891	best: 0.2281891 (100)	total: 5.27s	remaining: 1m 7s
200:	learn: 0.1543609	test: 0.2014988	best: 0.2014988 (200)	total: 10.3s	remaining: 1m 1s
300:	learn: 0.1330474	test: 0.1899977	best: 0.1899977 (300)	total: 15s	remaining: 54.9s
400:	learn: 0.1168727	test: 0.1817573	best: 0.1817573 (400)	total: 19.8s	remaining: 49.4s
500:	learn: 0.1045243	test: 0.1754421	best: 0.1754421 (500)	total: 24.6s	remaining: 44.1s
600:	learn: 0.0939051	test: 0.1703711	best: 0.1703711 (600)	total: 29.6s	remaining: 39.4s
700:	learn: 0.0862025	test: 0.1675898	best: 0.1675898 (700)	total: 34.3s	remaining: 34.2s
800:	learn: 0.0795372	test: 0.1653479	best: 0.1653148 (799)	total: 39.1s	remaining: 29.2s
900:	learn: 0.0733151	test: 0.1630350	best: 0.1629724 (898)	total: 43.9s	remaining: 24.3s
1000:	learn: 0.0685958	test: 0.1611894	best: 0.1611894 (1000)	total: 48.7s	remaining: 19.4s
1100:	learn: 

In [43]:
pred_1 = model_1.predict(X_test)
auc_1 = roc_auc_score(y_test, pred_1)

print("ROC AUC (товар/не товар):", auc_1)
print(classification_report(y_test, pred_1))

ROC AUC (товар/не товар): 0.9427433155041877
              precision    recall  f1-score   support

           0       0.93      0.96      0.94      1643
           1       0.96      0.93      0.94      1698

    accuracy                           0.94      3341
   macro avg       0.94      0.94      0.94      3341
weighted avg       0.94      0.94      0.94      3341



In [41]:
remaining_data = types_df.loc[types_df['Флаг'] == 0]

X_2 = np.vstack(remaining_data['embedding'].values)
y_2 = remaining_data['Тип']

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_2, y_2, test_size=0.3, random_state=42)
X_test_2, X_val_2, y_test_2, y_val_2 = train_test_split(X_test_2, y_test_2, test_size=0.3, random_state=42)

train_pool_2 = Pool(X_train_2, label=y_train_2)
test_pool_2 = Pool(X_val_2, label=y_val_2)

model_2 = CatBoostClassifier(iterations=2000,
                             learning_rate=0.03,
                             depth=6,
                             loss_function='Logloss',
                             verbose=100,
                             task_type='GPU')

model_2.fit(train_pool_2, eval_set=test_pool_2, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

0:	learn: 0.6811724	test: 0.6812324	best: 0.6812324 (0)	total: 49.4ms	remaining: 1m 38s
100:	learn: 0.3696441	test: 0.4216720	best: 0.4216720 (100)	total: 4.51s	remaining: 1m 24s
200:	learn: 0.2988099	test: 0.3868400	best: 0.3868400 (200)	total: 8.86s	remaining: 1m 19s
300:	learn: 0.2571016	test: 0.3720892	best: 0.3720892 (300)	total: 13.1s	remaining: 1m 13s
400:	learn: 0.2244452	test: 0.3610865	best: 0.3610865 (400)	total: 17.4s	remaining: 1m 9s
500:	learn: 0.1985866	test: 0.3522638	best: 0.3522638 (500)	total: 21.7s	remaining: 1m 4s
600:	learn: 0.1785279	test: 0.3462799	best: 0.3462799 (600)	total: 26.1s	remaining: 1m
700:	learn: 0.1622394	test: 0.3422019	best: 0.3421706 (698)	total: 30.3s	remaining: 56.1s
800:	learn: 0.1467131	test: 0.3380425	best: 0.3377701 (793)	total: 34.5s	remaining: 51.6s
900:	learn: 0.1339464	test: 0.3360253	best: 0.3360253 (900)	total: 38.8s	remaining: 47.3s
1000:	learn: 0.1226638	test: 0.3345238	best: 0.3344512 (999)	total: 43s	remaining: 42.9s
1100:	learn: 

In [42]:
pred_2 = model_2.predict(X_test_2)
auc_2 = roc_auc_score(y_test_2, pred_2)

print("ROC AUC (услуга/работа):", auc_2)
print(classification_report(y_test_2, pred_2))

ROC AUC (услуга/работа): 0.8450315405913699
              precision    recall  f1-score   support

           1       0.88      0.95      0.92      1126
           2       0.88      0.74      0.80       544

    accuracy                           0.88      1670
   macro avg       0.88      0.85      0.86      1670
weighted avg       0.88      0.88      0.88      1670



In [44]:
def pred(name):
    to_pred = pd.DataFrame({
    'Название': [name]
    })
    
    to_pred['embedding'] = to_pred['Название'].apply(lambda x: get_name_embedding(x, model, tokenizer))
    X = np.vstack(to_pred['embedding'].values)
    
    y_pred = model_1.predict(X)
    if y_pred == [1]:
        return 'Товар'
    
    y_pred_2 = model_2.predict(X)
    if y_pred_2 == [1]:
        return 'Услуга'
    return 'Работа'

In [45]:
%%time
pred('Звуковая карта Steinberg UR12')

CPU times: user 30.1 ms, sys: 2.09 ms, total: 32.2 ms
Wall time: 29.3 ms


'Товар'

In [46]:
%%time
uslugi = [
    'Влажная уборка всех помещений',
    'Чистка кухни и санузла',
    'Мытье окон и лоджий',
    'Уборка пыли с мебели и техники',
    'Вынос мусора',
    'Диагностика и устранение неисправностей',
    'Чистка от пыли и замена термопасты',
    'Переустановка операционной системы',
    'Установка программного обеспечения',
    'Восстановление данных',
    'Создание индивидуального образа',
    'Подбор места и времени съемки',
    'Обработка фотографий',
    'Печать фотокниг и фотоальбомов',
    'Разработка концепции свадьбы',
    'Выбор подрядчиков (ведущий, фотограф, видеооператор)',
    'Координация свадебного дня',
    'Разработка проекта озеленения',
    'Подбор растений',
    'Устройство газона',
    'Обустройство садовых дорожек',
    'Уход за садом',
    'Перевозка мебели и вещей',
    'Доставка товаров из магазинов',
    'Офисные переезды',
    'Перевозка негабаритных грузов'
]

res_uslugi = {x: '' for x in uslugi}

for u in uslugi:
    pr = pred(u)
    res_uslugi[u] = pr
    
res_uslugi

CPU times: user 910 ms, sys: 17.6 ms, total: 927 ms
Wall time: 887 ms


{'Влажная уборка всех помещений': 'Услуга',
 'Чистка кухни и санузла': 'Услуга',
 'Мытье окон и лоджий': 'Работа',
 'Уборка пыли с мебели и техники': 'Услуга',
 'Вынос мусора': 'Услуга',
 'Диагностика и устранение неисправностей': 'Услуга',
 'Чистка от пыли и замена термопасты': 'Услуга',
 'Переустановка операционной системы': 'Услуга',
 'Установка программного обеспечения': 'Услуга',
 'Восстановление данных': 'Работа',
 'Создание индивидуального образа': 'Услуга',
 'Подбор места и времени съемки': 'Услуга',
 'Обработка фотографий': 'Услуга',
 'Печать фотокниг и фотоальбомов': 'Работа',
 'Разработка концепции свадьбы': 'Услуга',
 'Выбор подрядчиков (ведущий, фотограф, видеооператор)': 'Услуга',
 'Координация свадебного дня': 'Услуга',
 'Разработка проекта озеленения': 'Работа',
 'Подбор растений': 'Товар',
 'Устройство газона': 'Работа',
 'Обустройство садовых дорожек': 'Работа',
 'Уход за садом': 'Услуга',
 'Перевозка мебели и вещей': 'Услуга',
 'Доставка товаров из магазинов': 'Работ

In [47]:
%%time
products = [
    'Шоколад Alpen Gold "Орео" 90 г',
    'Зубная паста Colgate Total 12 Чистая мята 100 мл',
    'Шампунь Pantene Pro-V "Густые и крепкие" 400 мл',
    'Мыло Dove "Красота и уход" 100 г',
    'Туалетная бумага Zewa Deluxe 3 слоя 4 рулона',
    'Стиральный порошок Ariel "Горный воздух" 3 кг',
    'Кетчуп Heinz 570 г',
    'Масло подсолнечное "Слобода" 1 л',
    'Крупа гречневая "Мистраль" 800 г',
    'Макароны "Макфа" 400 г',
    'Мука пшеничная "Алейка" 2 кг',
    'Сахар "Русский сахар" 1 кг',
    'Соль "Экстра" 1 кг',
    'Чай черный Greenfield "Летний сад" 25 пакетиков',
    'Кофе Jacobs Monarch "Original" 250 г',
    'Сок J7 "Яблоко" 1 л',
    'Вода "Святой источник" 0.5 л',
    'Хлеб "Дарница" 400 г',
    'Яблоки "Гренни Смит" 1 кг',
    'Помидоры "Сливовка" 1 кг'
]

res_products = {x: '' for x in products}

for p in products:
    pr = pred(p)
    res_products[p] = pr
    
res_products

CPU times: user 473 ms, sys: 4.63 ms, total: 477 ms
Wall time: 461 ms


{'Шоколад Alpen Gold "Орео" 90 г': 'Товар',
 'Зубная паста Colgate Total 12 Чистая мята 100 мл': 'Товар',
 'Шампунь Pantene Pro-V "Густые и крепкие" 400 мл': 'Товар',
 'Мыло Dove "Красота и уход" 100 г': 'Товар',
 'Туалетная бумага Zewa Deluxe 3 слоя 4 рулона': 'Товар',
 'Стиральный порошок Ariel "Горный воздух" 3 кг': 'Товар',
 'Кетчуп Heinz 570 г': 'Товар',
 'Масло подсолнечное "Слобода" 1 л': 'Товар',
 'Крупа гречневая "Мистраль" 800 г': 'Товар',
 'Макароны "Макфа" 400 г': 'Товар',
 'Мука пшеничная "Алейка" 2 кг': 'Товар',
 'Сахар "Русский сахар" 1 кг': 'Товар',
 'Соль "Экстра" 1 кг': 'Товар',
 'Чай черный Greenfield "Летний сад" 25 пакетиков': 'Товар',
 'Кофе Jacobs Monarch "Original" 250 г': 'Товар',
 'Сок J7 "Яблоко" 1 л': 'Товар',
 'Вода "Святой источник" 0.5 л': 'Товар',
 'Хлеб "Дарница" 400 г': 'Товар',
 'Яблоки "Гренни Смит" 1 кг': 'Товар',
 'Помидоры "Сливовка" 1 кг': 'Товар'}

In [48]:
%%time
works = [
    'Программист-разработчик Python',
    'Веб-дизайнер',
    'Маркетинг-менеджер',
    'Финансовый аналитик',
    'HR-специалист',
    'Контент-менеджер',
    'Медицинская сестра',
    'Преподаватель английского языка',
    'Инженер-строитель',
    'Продавец-консультант',
    'Администратор баз данных',
    'Мобильный разработчик',
    'Архитектор интерьера',
    'Системный аналитик',
    'Инженер-исследователь',
    'Административный помощник',
    'Психотерапевт',
    'Автомеханик',
    'Продуктовый менеджер',
    'Дизайнер UX/UI'
]

res_works = {x: '' for x in works}

for p in works:
    pr = pred(p)
    res_works[p] = pr
    
res_works

CPU times: user 729 ms, sys: 13 ms, total: 742 ms
Wall time: 718 ms


{'Программист-разработчик Python': 'Услуга',
 'Веб-дизайнер': 'Услуга',
 'Маркетинг-менеджер': 'Услуга',
 'Финансовый аналитик': 'Услуга',
 'HR-специалист': 'Услуга',
 'Контент-менеджер': 'Услуга',
 'Медицинская сестра': 'Услуга',
 'Преподаватель английского языка': 'Услуга',
 'Инженер-строитель': 'Услуга',
 'Продавец-консультант': 'Услуга',
 'Администратор баз данных': 'Услуга',
 'Мобильный разработчик': 'Работа',
 'Архитектор интерьера': 'Услуга',
 'Системный аналитик': 'Услуга',
 'Инженер-исследователь': 'Услуга',
 'Административный помощник': 'Услуга',
 'Психотерапевт': 'Услуга',
 'Автомеханик': 'Услуга',
 'Продуктовый менеджер': 'Услуга',
 'Дизайнер UX/UI': 'Товар'}

In [49]:
model_1.save_model('model_1')
model_2.save_model('model_2')